In [79]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch

In [80]:
df = pd.read_csv("./data/pulsar_stars.csv",nrows=4000)

In [81]:
df

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
3995,108.421875,49.875252,0.356194,0.140122,11.224916,43.600214,3.942828,14.564314,0
3996,101.898438,50.420240,0.461758,0.376136,3.197324,23.872246,8.232433,70.314547,0
3997,112.976562,46.050154,0.458158,0.435297,3.989130,22.797107,6.788768,50.696445,0
3998,99.515625,47.455733,0.451288,0.465505,14.775084,48.659888,3.245646,9.195834,0


In [82]:
y_all=df.iloc[:,-1]
X_all = df.iloc[:,0:-1]

In [83]:
y_all

0       0
1       0
2       0
3       0
4       0
       ..
3995    0
3996    0
3997    0
3998    0
3999    0
Name: target_class, Length: 4000, dtype: int64

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all,test_size =0.2, random_state = 0, stratify=y_all)

In [85]:
standardScaler = StandardScaler()
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.transform(X_test)

In [86]:
X_train

array([[-0.57311801, -1.34087715,  0.07956357, ..., -0.11578355,
        -0.22533227, -0.4479921 ],
       [ 0.63820629,  0.18951931, -0.58787324, ...,  0.5662394 ,
        -0.69795533, -0.72124099],
       [ 0.90108944,  0.35385443, -0.38575383, ..., -0.39820254,
         0.38322914,  0.06422995],
       ...,
       [-0.56824981, -0.07057353, -0.12017248, ..., -0.32801744,
        -0.14443882, -0.3276834 ],
       [ 0.36243672, -0.24088214, -0.53858603, ..., -0.448954  ,
         0.39133959,  0.06282571],
       [-2.03157539, -0.39130943,  1.85232287, ...,  0.67293441,
        -0.89520212, -0.786471  ]])

## torch

In [95]:
X_train[0]
X_train.shape[0]
len(X_train)

3200

In [88]:
y_train.values

array([0, 0, 0, ..., 0, 0, 1])

In [89]:
x=torch.tensor(X_train,dtype=float)
y=torch.tensor(y_train.values,dtype=float)

weights=torch.randn((X_train.shape[-1],128),dtype=float,requires_grad=True)
biases=torch.randn(128,dtype=float,requires_grad=True)
weights2=torch.randn((128,1),dtype=float,requires_grad=True)
biases2=torch.randn(1,dtype=float,requires_grad=True)

learning_rate=0.001
losses=[]

for i in range(1000):
    hidden=x.mm(weights)+biases # ax+b
    hidden=torch.relu(hidden)
    predictions=hidden.mm(weights2)+biases2
    
    loss=torch.mean((predictions-y)**2)
    losses.append(loss.data.numpy())

    if i% 100==0:
        print('loss:',loss)
    loss.backward()

    weights.data.add_(-learning_rate*weights.grad.data)
    biases.data.add_(-learning_rate*biases.grad.data)
    weights2.data.add_(-learning_rate*weights2.grad.data)
    biases2.data.add_(-learning_rate*biases2.grad.data)

    weights.grad.data.zero_()
    biases.grad.data.zero_()
    weights2.grad.data.zero_()
    biases2.grad.data.zero_()


loss: tensor(556.8202, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(8.1121, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(4.4045, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(3.1848, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(2.5434, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(2.1247, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.8237, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.5969, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.4218, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(1.2849, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [90]:
x
y

tensor([0., 0., 0.,  ..., 0., 0., 1.], dtype=torch.float64)

## net

In [93]:
input_size=X_train.shape[-1]
hidden_size=128
output=1
batch_size=16
my_nn= torch.nn.Sequential(
    torch.nn.Linear(input_size,hidden_size),# 8,128
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_size,output) # 128,1
    )
cost=torch.nn.MSELoss(reduction='mean')
optimizer=torch.optim.Adam(my_nn.parameters(),lr=0.001)

In [99]:
losses=[]
for i in range(1000):
    batch_loss=[]
    for start in range(0,len(X_train),batch_size):
        end=start+batch_size if start+batch_size<len(X_train) else len(X_train)
        xx= torch.tensor(X_train[start:end],dtype=torch.float,requires_grad=True)
        yy= torch.tensor(y_train.values[start:end],dtype=torch.float,requires_grad=True)
        prediction=my_nn(xx)
        loss=cost(prediction,yy)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        batch_loss.append(loss.data.numpy())

    if i% 100==0:
        losses.append(np.mean(batch_loss))
        print(i,np.mean(batch_loss))
   

0 0.12035042
100 0.11877444
200 0.117976435
300 0.117432944
400 0.11706591
500 0.11676893
600 0.11654892
700 0.11639848
800 0.1163039
900 0.11624064


In [100]:
X_test_tensor = torch.tensor(X_test,dtype=torch.float)
predict=my_nn(X_test_tensor).data.numpy()